In [1]:
from netCDF4 import Dataset
import numpy as np 
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy.ma as ma
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
import pyart
import matplotlib.lines as mlines
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import pickle
from datetime import datetime, timedelta
from metpy.interpolate.grid import natural_neighbor_to_grid
from metpy.interpolate import interpolate_to_points
from metpy.interpolate import interpolate_to_grid
import csv


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



C:\Users\matts\anaconda3\envs\spork2022\lib\site-packages\xarray\core\pycompat.py:22: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion(duck_array_module.__version__)
C:\Users\matts\anaconda3\envs\spork2022\lib\site-packages\xarray\core\pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion("0.0.0")
C:\Users\matts\anaconda3\envs\spork2022\lib\site-packages\xarray\core\pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion("0.0.0")
C:\Users\matts\anaconda3\envs\spork2022\lib\site-packages\setuptools\_distutils\version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\matts\anaconda3\envs\spork2022\lib\site-packages\setuptools\_distutils\version

In [2]:
# MIT License

# Copyright (c) 2017 thunderhoser

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

import numpy
from scipy.interpolate import interp1d


def _run_pmm_one_variable(
        input_matrix, max_percentile_level=100):
    """Applies PMM to one variable.
    E = number of examples (realizations over which to average)
    :param input_matrix: numpy array.  The first axis must have length E.  Other
        axes are assumed to be spatial dimensions.  Thus, input_matrix[i, ...]
        is the spatial field for the [i]th example.
    :param max_percentile_level: Maximum percentile.  No output value will
        exceed the [q]th percentile of `input_matrix`, where q =
        `max_percentile_level`.  Similarly, no output value will be less than
        the [100 - q]th percentile of `input_matrix`.
    :return: mean_field_matrix: numpy array of probability-matched means.  Will
        have the same dimensions as `input_matrix`, except without the first
        axis.  For example, if `input_matrix` is E x 32 x 32 x 12, this will be
        32 x 32 x 12.
    """

    # Pool values over all dimensions and remove extremes.
    pooled_values = numpy.ravel(input_matrix)
    pooled_values = numpy.sort(pooled_values)

    max_pooled_value = numpy.percentile(pooled_values, max_percentile_level)
    pooled_values = pooled_values[pooled_values <= max_pooled_value]

    min_pooled_value = numpy.percentile(
        pooled_values, 100 - max_percentile_level)
    pooled_values = pooled_values[pooled_values >= min_pooled_value]

    # Find ensemble mean at each grid point.
    mean_field_matrix = numpy.mean(input_matrix, axis=0)
    mean_field_flattened = numpy.ravel(mean_field_matrix)

    # At each grid point, replace ensemble mean with the same percentile from
    # pooled array.
    pooled_value_percentiles = numpy.linspace(
        0, 100, num=len(pooled_values), dtype=float)
    mean_value_percentiles = numpy.linspace(
        0, 100, num=len(mean_field_flattened), dtype=float)

    sort_indices = numpy.argsort(mean_field_flattened)
    unsort_indices = numpy.argsort(sort_indices)

    interp_object = interp1d(
        pooled_value_percentiles, pooled_values, kind='linear',
        bounds_error=True, assume_sorted=True)

    mean_field_flattened = interp_object(mean_value_percentiles)
    mean_field_flattened = mean_field_flattened[unsort_indices]
    mean_field_matrix = numpy.reshape(
        mean_field_flattened, mean_field_matrix.shape)

    return mean_field_matrix

In [3]:
#Rotation code from https://stackoverflow.com/questions/29708840/rotate-meshgrid-with-numpy
def DoRotation(xspan, yspan, RotRad=0):
    """Generate a meshgrid and rotate it by RotRad radians."""

    # Clockwise, 2D rotation matrix
    RotMatrix = np.array([[np.cos(RotRad),  np.sin(RotRad)],
                          [-np.sin(RotRad), np.cos(RotRad)]])

    x, y = np.meshgrid(xspan, yspan)
    return np.einsum('ji, mni -> jmn', RotMatrix, np.dstack([x, y]))

In [4]:
NewCases = np.genfromtxt('SPORK_rerun_new.csv', delimiter=',', usecols=(0,2,3,4,7,17),skip_header=1, dtype=int)
NewCases1 = np.genfromtxt('SPORK_rerun_new.csv', delimiter=',', usecols=(0,2,3,4,7,17), dtype=str, skip_header=1)

print(NewCases[0,:])
print(NewCases1[0,:])

ALL_sites = NewCases1[:,0]
ALL_years = NewCases[:,1]
ALL_months = NewCases[:,2]
ALL_days = NewCases[:,3]
ALL_hours = NewCases[:,4]
ffds = NewCases[:,5]
print(ffds)
#ALL_TN = pickle.load(open('ALLVARTor_Non.pkl', 'rb'))

[  -1 2021    6   19    0  220]
['KIND' '2021' '6' '19' '0' '220']
[220 160 190 145 200 140 160 180 110 170 120 170 165 140 150 140 140 170
 170 150 150 130 150 130 140 190 180 180 180 180 130  70 170 200 200 190
 190 120 130 140 200 200 180 220 190 180 190 160 160 150 160 150 160 180
 140 150 140 170 150 160 180 180 210 230 190 230 180 180 150 130 190 240
 170 130 190 160 130 150 180 180 190 190 180 230 150 150 200 230 170 190
 140 150 150 220 160 190 180 180 160 170 180 150 160 150]


In [5]:
def rotate_stamp(ffd, ncfile, xoff, yoff, st_2a, st_2b):
    ref = ncfile.variables['REFL'][:]
    kdp = ncfile.variables['KDP'][:]
    zdr = ncfile.variables['ZDR'][:]
    cc = ncfile.variables['CC'][:]
    zdr[ref < 20] = np.nan
    zdr[cc < 0.9] = np.nan
    kdp[ref < 20] = np.nan
    kdp[cc < 0.9] = np.nan
    nzdr = ncfile.variables['NZDR'][:]
    nzdr[ref[:,4,:,:] < 20] = 0.0
    nzdr[cc[:,4,:,:] < 0.95] = 0.0
    zdrd = ncfile.variables['ZDRD'][:]
    rot = ncfile.variables['ROT'][:]
    times = ncfile.variables['Times'][:]
    azim = ncfile.variables['azim'][:]
    vel = ncfile.variables['VEL'][:]
    
    try:
        ncfile7 = Dataset('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(int(ALL_days[i]))+str(ALL_sites[i])+'.nc')
        times = ncfile7.variables['Times'][:]

    except:
        ncfile7 = Dataset('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(int(ALL_days[i])-1)+str(ALL_sites[i])+'.nc')
        times = ncfile7.variables['Times'][:]

    st_1 = datetime(int(ALL_years[i]), int(ALL_months[i]), int(ALL_days[i]), int(ALL_hours[i]), 0)-timedelta(minutes=30)
    print(st_1)
    time_diffs = []
    for j in range(len(times)):

        time_diff = (datetime.fromtimestamp(times[j]) - st_1).total_seconds()
        time_diffs.append(time_diff)
    time_array = np.asarray(time_diffs)  
    time_array[time_array<0] = 9999
    
    #Set up logic to clip the time array to remove times when the storm is
    #too far from the radar. Set the time_diffs at those times to be 9999.
    #Removing times before the start time
    if st_2a:
        time_diffs2 = []
        for j in range(len(times)):
            time_diff2 = (datetime.fromtimestamp(times[j]) - st_2a).total_seconds()
            time_diffs2.append(time_diff2)
        time_array2 = np.asarray(time_diffs2)  
        time_array[time_array2<0] = 9999
        
    #Removing times after the end time
    if st_2b:
        time_diffs3 = []
        for j in range(len(times)):
            time_diff3 = (datetime.fromtimestamp(times[j]) - st_2b).total_seconds()
            time_diffs3.append(time_diff3)
        time_array3 = np.asarray(time_diffs3)  
        time_array[time_array3>0] = 9999
    
    print(time_array)
    print(time_diffs)
    print(np.where(time_array < 3600))
    print(ref[time_array < 3600, :, :, :].shape[0], 'times')

    ref_mn = ref[time_array < 3600, :, :, :]
    zdr_mn = zdr[time_array < 3600, :, :, :]
    kdp_mn = kdp[time_array < 3600, :, :, :]
    cc_mn = cc[time_array < 3600, :, :, :]
    rot_mn = rot[time_array < 3600, :, :, :]
    zdrd_mn = zdrd[time_array < 3600, :, :]
    nzdr_mn = nzdr[time_array < 3600, :, :]

    ref_st = np.nanmean(ref_mn, axis=0)
    zdr_st = np.nanmean(zdr_mn, axis=0)
    kdp_st = np.nanmean(kdp_mn, axis=0)
    cc_st = np.nanmean(cc_mn, axis=0)
    rot_st = np.nanmean(rot_mn, axis=0)
    zdrd_st = np.nanmean(zdrd_mn, axis=0)
    nzdr_st = np.nanmean(nzdr_mn, axis=0)

    nzdr_mn[np.isnan(nzdr_mn)] = 0.0
    zdrd_mn[np.isnan(zdrd_mn)] = 0.0
    zdr_mn[np.isnan(zdr_mn)] = 0.0
    kdp_mn[np.isnan(kdp_mn)] = 0.0
    cc_mn[np.isnan(cc_mn)] = 0.0
    rot_mn[ref_mn < 20.0] = 0.0

    ref_pm = _run_pmm_one_variable(ref_mn)
    rot_pm = _run_pmm_one_variable(rot_mn)
    zdrd_pm =  _run_pmm_one_variable(zdrd_mn)
    nzdr_pm =  _run_pmm_one_variable(nzdr_mn)
    zdr_pm =  _run_pmm_one_variable(zdr_mn)
    kdp_pm =  _run_pmm_one_variable(kdp_mn)
    cc_pm =  _run_pmm_one_variable(cc_mn)
    for q in range(zdr_mn.shape[0]):
        print(np.max(rot_mn[q,12,:,:]))
    print(ref_mn.shape)
    xr = np.arange(-70,70,1)
    xra1, yra1 = DoRotation(xr, xr, RotRad=np.deg2rad((ffd)-180)*-1)
    xri, yri, Z_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(ref_pm[4,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, rot1_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(rot_pm[4,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, rot3_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(rot_pm[12,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, rot5_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(rot_pm[20,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, rot7_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(rot_pm[28,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, zdrd_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(zdrd_pm[:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, zdr_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(zdr_pm[4,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, nzdr_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(nzdr_pm[:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, kdp_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(kdp_pm[4,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, cc_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(cc_pm[4,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    return xri, yri, Z_interpi, rot1_interpi, rot3_interpi, rot5_interpi, rot7_interpi, zdrd_interpi, nzdr_interpi, zdr_interpi, kdp_interpi, cc_interpi

In [6]:
#Get the list of meso locations
meso_locs = np.genfromtxt('MesocycloneLocationsPass3.csv', delimiter=',', skip_header=1)
meso_x = meso_locs[206:,3]
meso_y = meso_locs[206:,4]

In [7]:
print(len(meso_x))
print(len(ALL_years))

203
104


In [12]:
#Generate the file names for the postage stamps
j=1
#for i in range(len(ALL_years)):
#for i in np.arange(1,104,1):
for i in [68]:
    try:
        print('D:/SPORK_AC/SPORK_AC'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc')
        ncfile = Dataset('D:/SPORK_AC/SPORK_AC'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc')
    except:
        try:
            ncfile = Dataset('D:/SPORK_AC/SPORK_AC'+str(ALL_years[i])+str(ALL_months[i])+str(int(ALL_days[i])-1)+str(ALL_sites[i])+'.nc')
        except:
            print('SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc is missing!')
            print(j)
            j=j+1
    try:
        #rotated_data = rotate_stamp(ffds[i], ncfile, meso_x[i], meso_y[i])
        rotated_data = rotate_stamp(ffds[i], ncfile, meso_x[i], meso_y[i], st_2a=None, st_2b=datetime(2022, 5, 4, 23, 22))

        
        with open('SPORK_SHIFT_AC3'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.pkl', 'wb') as f:
            pickle.dump(rotated_data, f)
    except:
        print('SPORK_40_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc failed!')
        

D:/SPORK_AC/SPORK_AC202254KAMA.nc
2022-05-04 22:30:00
[9999. 9999. 9999. 9999.  226.  472.  731.  977. 1245. 1621. 2030. 2425.
 2792. 9999. 9999. 9999. 9999.]
[-859.0, -577.0, -295.0, -36.0, 226.0, 472.0, 731.0, 977.0, 1245.0, 1621.0, 2030.0, 2425.0, 2792.0, 3159.0, 3555.0, 3965.0, 4363.0]
(array([ 4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64),)
9 times


C:\Users\matts\AppData\Local\Temp\ipykernel_34040\2188689262.py:71: RuntimeWarning: Mean of empty slice
  zdr_st = np.nanmean(zdr_mn, axis=0)
C:\Users\matts\AppData\Local\Temp\ipykernel_34040\2188689262.py:72: RuntimeWarning: Mean of empty slice
  kdp_st = np.nanmean(kdp_mn, axis=0)
C:\Users\matts\AppData\Local\Temp\ipykernel_34040\2188689262.py:75: RuntimeWarning: Mean of empty slice
  zdrd_st = np.nanmean(zdrd_mn, axis=0)


0.008989215873783587
0.009589099958929759
0.009737348459797023
0.007647672893665318
0.007887307866553188
0.007593793302889412
0.01093955169285279
0.004811864171043495
0.006256046815425873
(9, 41, 140, 140)


In [ ]:
plt.figure(1, figsize=(24,24))
xr2 = np.arange(0, len(rotated_data[0]), 1)
reflevs = np.arange(20, 70, 5)
zdrdlev = np.arange(1, 10, 1)
rotlev = np.arange(0.002,0.01, 0.002)
# plt.contourf(xr, xr, ref_st[4, :, :], reflevs, cmap=pyart.graph.cm_colorblind.HomeyerRainbow)
# plt.contour(xr, xr, zdrd_st[:, :]/4, zdrdlev, cmap=plt.cm.viridis, linewidths=5)
plt.contourf(rotated_data[0], rotated_data[1], rotated_data[2], reflevs, cmap=pyart.graph.cm_colorblind.HomeyerRainbow)
plt.contourf(rotated_data[0], rotated_data[1], rotated_data[7]/4, zdrdlev, cmap=plt.cm.viridis)
plt.contour(rotated_data[0], rotated_data[1], rotated_data[3], rotlev, colors='r', linewidths=4)
plt.contour(rotated_data[0], rotated_data[1], rotated_data[4], rotlev, colors='purple', linewidths=4)
plt.contour(rotated_data[0], rotated_data[1], rotated_data[5], rotlev, colors='blue', linewidths=4)

In [ ]:
from pyproj import Geod
import numpy as np
from datetime import datetime, timedelta

#Create geod object for later distance and area calculations
g = Geod(ellps='sphere')
def radar_motion(start_time, end_time, sloni, slati, slone, slate):

    stormdte = (np.datetime64(end_time) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    stormdtce = datetime.utcfromtimestamp(stormdte)
    stormdti = (np.datetime64(start_time) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    stormdtci = datetime.utcfromtimestamp(stormdti)

    distance_track = g.inv(sloni, slati,
                           slone, slate)
    dist_track = distance_track[2]                                        
    if distance_track[1] < 0:
        back = distance_track[1] + 360
    else:
        back = distance_track[1]
    storm_dur = stormdtce-stormdtci
    storm_sec = storm_dur.seconds
    speed = dist_track/storm_sec
    direc = back
    return speed, direc

In [ ]:
#Get observed motions for all of these cases
#Generate the file names for the postage stamps
j=1
new_speeds = []
new_directions = []

for i in range(len(ALL_years)):
#for i in np.arange(201,207,1):
#for i in [104]:
    try:
        print('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc')
        ncfile = Dataset('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc')
    except:
        try:
            ncfile = Dataset('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(int(ALL_days[i])-1)+str(ALL_sites[i])+'.nc')
        except:
            print('SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc is missing!')
            print(j)
            j=j+1
    case_loni = ncfile.variables['Lons'][:,70,70] 
    case_lati = ncfile.variables['Lats'][:,70,70] 
    case_timesi = ncfile.variables['Times'][:] 
    #print(case_loni, case_lati, case_timesi)
    
    st_1 = datetime(int(ALL_years[i]), int(ALL_months[i]), int(ALL_days[i]), int(ALL_hours[i]), 0)-timedelta(minutes=30)
    #print(st_1)
    time_diffs = []
    for j in range(len(case_timesi)):

        time_diff = (datetime.fromtimestamp(case_timesi[j]) - st_1).total_seconds()
        time_diffs.append(time_diff)
    time_array = np.asarray(time_diffs)  
    time_array[time_array<0] = 9999
    #print(time_array)
    #print(time_diffs)
    #print(np.where(time_array < 3600))
    
    case_lon = case_loni[time_array < 3600]
    case_lat = case_lati[time_array < 3600]
    case_times = case_timesi[time_array < 3600]
    
    Xlon = np.array(case_lon)
    Ylat = np.array(case_lat)
    X_obs1 = Xlon[0]
    X_obs2 = Xlon[-1]
    X_mean = np.mean(Xlon)
    Y_mean = np.mean(Ylat)
    num = 0
    den = 0
    for i in range (len(Xlon)):
        num += (Xlon[i] - X_mean)*(Ylat[i] - Y_mean)
        den += (Xlon[i] - X_mean)**2
    m = num / den
    c = Y_mean - m*X_mean
    Y_pred2 = m*X_obs2 + c
    Y_pred1 = m*X_obs1 + c
    #print("X_obs1, Y_pred1 =", X_obs1, Y_pred1, "; X_obs2, Y_pred2 =", X_obs2, Y_pred2)
    
    st_speed, st_direction = radar_motion(datetime.fromtimestamp(case_times[0]), datetime.fromtimestamp(case_times[-1]), X_obs1, Y_pred1, X_obs2, Y_pred2)
    
    print(st_speed, st_direction)
    
    new_speeds.append(st_speed)
    new_directions.append(st_direction)

In [ ]:
print(ncfile.variables['Lons'][:,70,70])

In [ ]:
print(Xlon)
print(Xlon[0])
print(Xlon[-1])

In [ ]:
print(case_times[0])
timestamp = case_times[0]
dt_object = datetime.fromtimestamp(timestamp)
print(dt_object)

In [ ]:
print(new_speeds)
print(new_directions)

In [ ]:
efilename = 'NewObsMotions.csv'
with open(efilename, mode='w') as cal_file:
    calWriter = csv.writer(cal_file, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)

for i in range(len(ALL_years)):

    with open(efilename, mode='a') as cal_file:
            calWriter = csv.writer(cal_file, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
            calWriter.writerow([ALL_years[i], ALL_months[i], ALL_days[i], ALL_hours[i], new_speeds[i], new_directions[i]])